In [123]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


#**Data**

In [124]:
food = pd.read_csv('/content/gdrive/My Drive/BMKG/Food.csv')
compound = pd.read_csv('/content/gdrive/My Drive/BMKG/Compound.csv')
nutrient = pd.read_csv('/content/gdrive/My Drive/BMKG/Nutrient.csv')
content = pd.read_csv('/content/gdrive/My Drive/BMKG/Content.csv')
health_effect = pd.read_csv('/content/gdrive/My Drive/BMKG/HealthEffect.csv')
compounds_health_effect = pd.read_csv('/content/gdrive/My Drive/BMKG/CompoundsHealthEffect.csv')
flavor = pd.read_csv('/content/gdrive/My Drive/BMKG/Flavor.csv')
compounds_flavor = pd.read_csv('/content/gdrive/My Drive/BMKG/CompoundsFlavor.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3,4,5,6,12,13,14,15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4,5,6,7,8,9,13,14,21,22,24) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [125]:
food = food.replace('NA',np.nan)
food = food[['id', 'name', 'food_group', 'food_subgroup']]
food = food.dropna()

compound = compound.replace('NA',np.nan)
compound = compound[['id', 'name']]
compound = compound.dropna()

nutrient = nutrient.replace('NA',np.nan)
nutrient = nutrient[['id', 'name']]
nutrient = nutrient.dropna()

content = content.replace('NA',np.nan)
content = content[['source_id', 'source_type', 'food_id', 'orig_content', 'orig_unit']]
content_compound = content[(content['source_type'] == 'Compound')]
content_nutrient = content[(content['source_type'] == 'Nutrient')]
content_compound = content_compound.dropna()
content_nutrient = content_nutrient.dropna()

health_effect = health_effect.replace('NA',np.nan)
health_effect = health_effect[['id', 'name']]
health_effect = health_effect.dropna()

compounds_health_effect = compounds_health_effect.replace('NA',np.nan)
compounds_health_effect = compounds_health_effect[['compound_id', 'health_effect_id']]
compounds_health_effect = compounds_health_effect.dropna()

flavor = flavor.replace('NA',np.nan)
flavor = flavor[['id', 'name']]
flavor = flavor.dropna()

compounds_flavor = compounds_flavor.replace('NA',np.nan)
compounds_flavor = compounds_flavor[['compound_id', 'flavor_id']]
compounds_flavor = compounds_flavor.dropna()

In [126]:
content_compound = content_compound[content_compound.orig_content != 0]
content_nutrient = content_nutrient[content_nutrient.orig_content != 0]
content_compound = content_compound.drop_duplicates(subset=['source_id', 'food_id'])
content_nutrient = content_nutrient.drop_duplicates(subset=['source_id', 'food_id'])

In [127]:
#Food-Compound
for i in range(len(food)):
  food_id = food['id'].iloc[i]
  food_name = food[food['id'] == food_id]['name'].iloc[0]
  food_group = food[food['id'] == food_id]['food_group'].iloc[0]
  food_subgroup = food[food['id'] == food_id]['food_subgroup'].iloc[0]
  index = content_compound[content_compound['food_id'] == food_id].index
  content_compound.at[index, 'food_name'] = food_name
  content_compound.at[index, 'food_group'] = food_group
  content_compound.at[index, 'food_subgroup'] = food_subgroup

for i in range(len(compound)):
  compound_id = compound['id'].iloc[i]
  compound_name = compound[compound['id'] == compound_id]['name'].iloc[0]
  index = content_compound[(content_compound['source_id'] == compound_id)].index
  content_compound.at[index, 'source_name'] = compound_name

In [128]:
#Food-Nutrient
for i in range(len(food)):
  food_id = food['id'].iloc[i]
  food_name = food[food['id'] == food_id]['name'].iloc[0]
  food_group = food[food['id'] == food_id]['food_group'].iloc[0]
  food_subgroup = food[food['id'] == food_id]['food_subgroup'].iloc[0]
  index = content_nutrient[content_nutrient['food_id'] == food_id].index
  content_nutrient.at[index, 'food_name'] = food_name
  content_nutrient.at[index, 'food_group'] = food_group
  content_nutrient.at[index, 'food_subgroup'] = food_subgroup

for i in range(len(nutrient)):
  nutrient_id = nutrient['id'].iloc[i]
  nutrient_name = nutrient[nutrient['id'] == nutrient_id]['name'].iloc[0]
  index = content_nutrient[(content_nutrient['source_id'] == nutrient_id)].index
  content_nutrient.at[index, 'source_name'] = nutrient_name

In [129]:
#Compound-Health Effect
for i in range(len(health_effect)):
  health_effect_id = health_effect['id'].iloc[i]
  health_effect_name = health_effect[health_effect['id'] == health_effect_id]['name'].iloc[0]
  index = compounds_health_effect[compounds_health_effect['health_effect_id'] == health_effect_id].index
  compounds_health_effect.at[index, 'health_effect_name'] = health_effect_name

for i in range(len(compound)):
  compound_id = compound['id'].iloc[i]
  compound_name = compound[compound['id'] == compound_id]['name'].iloc[0]
  index = compounds_health_effect[compounds_health_effect['compound_id'] == compound_id].index
  compounds_health_effect.at[index, 'compound_name'] = compound_name

In [130]:
#Compound-Flavor
for i in range(len(flavor)):
  flavor_id = flavor['id'].iloc[i]
  flavor_name = flavor[flavor['id'] == flavor_id]['name'].iloc[0]
  index = compounds_flavor[compounds_flavor['flavor_id'] == flavor_id].index
  compounds_flavor.at[index, 'flavor_name'] = flavor_name

for i in range(len(compound)):
  compound_id = compound['id'].iloc[i]
  compound_name = compound[compound['id'] == compound_id]['name'].iloc[0]
  index = compounds_flavor[compounds_flavor['compound_id'] == compound_id].index
  compounds_flavor.at[index, 'compound_name'] = compound_name

# **Nutri-Scores**

Preprocessing that needs to be done to compute the Nutri-Scores:

- convert mg/100g to g/100g

- convert kcal/100g to kJ/100g

- EXCEPT SODIUM !!!

In [131]:
content_nutrient.orig_unit = content_nutrient.orig_unit.replace('mg/100 g', 'mg/100g')
content_compound.orig_unit = content_compound.orig_unit.replace('mg/100 g', 'mg/100g')

In [132]:
for i in range(len(content_nutrient)):
  if content_nutrient['orig_unit'].iloc[i] == 'mg/100g':
    content_nutrient['orig_content'].iloc[i] /= 1000
    content_nutrient['orig_unit'].iloc[i] = 'g/100g'
  elif content_nutrient['orig_unit'].iloc[i] == 'kcal/100g':
    content_nutrient['orig_content'].iloc[i] *= 4.184
    content_nutrient['orig_unit'].iloc[i] = 'kJ/100g'

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [133]:
target = [2, 3, 4, 5, 38]
content_nutrient[content_nutrient['source_id'].isin(target)]['orig_unit'].unique()

array(['g/100g', 'kJ/100g'], dtype=object)

In [134]:
target = [3524]
content_compound[content_compound['source_id'].isin(target)]['orig_unit'].unique()

array(['mg/100g'], dtype=object)

In [135]:
def scoreEnergy(energy):
  if energy <= 335:
    return 0
  elif energy > 3350:
    return 10
  elif energy > 3015:
    return 9
  elif energy > 2680:
    return 8
  elif energy > 2345:
    return 7
  elif energy > 2010:
    return 6
  elif energy > 1675:
    return 5
  elif energy > 1340:
    return 4
  elif energy > 1005:
    return 3
  elif energy > 670:
    return 2
  elif energy > 335:
    return 1

def scoreSugar(carbohydrates, fibers):
  sugar = carbohydrates - fibers
  if sugar <= 4.5:
    return 0
  elif sugar > 45:
    return 10
  elif sugar > 40:
    return 9
  elif sugar > 36:
    return 8
  elif sugar > 31:
    return 7
  elif sugar > 27:
    return 6
  elif sugar > 22.5:
    return 5
  elif sugar > 18:
    return 4
  elif sugar > 13.5:
    return 3
  elif sugar > 9:
    return 2
  elif sugar > 4.5:
    return 1

def scoreSaturatedFattyAcis(saturatedFattyAcids):
  if saturatedFattyAcids <= 1:
    return 0
  elif saturatedFattyAcids > 10:
    return 10
  elif saturatedFattyAcids > 9:
    return 9
  elif saturatedFattyAcids > 8:
    return 8
  elif saturatedFattyAcids > 7:
    return 7
  elif saturatedFattyAcids > 6:
    return 6
  elif saturatedFattyAcids > 5:
    return 5
  elif saturatedFattyAcids > 4:
    return 4
  elif saturatedFattyAcids > 3:
    return 3
  elif saturatedFattyAcids > 2:
    return 2
  elif saturatedFattyAcids > 1:
    return 1

def scoreSodium(sodium):
  if sodium <= 90:
    return 0
  elif sodium > 900:
    return 10
  elif sodium > 810:
    return 9
  elif sodium > 720:
    return 8
  elif sodium > 630:
    return 7
  elif sodium > 540:
    return 6
  elif sodium > 450:
    return 5
  elif sodium > 360:
    return 4
  elif sodium > 270:
    return 3
  elif sodium > 180:
    return 2
  elif sodium > 90:
    return 1

def scoreFibers(fibers):
  if fibers <= 0.9:
    return 0
  elif fibers > 4.7:
    return -5
  elif fibers > 3.7:
    return -4
  elif fibers > 2.8:
    return -3
  elif fibers > 1.9:
    return -2
  elif fibers > 0.9:
    return -1

def scoreProteins(proteins):
  if proteins <= 1.6:
    return 0
  elif proteins > 8:
    return -5
  elif proteins > 6.4:
    return -4
  elif proteins > 4.8:
    return -3
  elif proteins > 3.2:
    return -2
  elif proteins > 1.6:
    return -1

def nutriScoreFoods(nutri_score):
  if nutri_score >= -15 and nutri_score <= -1:
    return 'A'
  elif nutri_score >= 0 and nutri_score <= 2:
    return 'B'
  elif nutri_score >= 3 and nutri_score <= 10:
    return 'C'
  elif nutri_score >= 11 and nutri_score <= 18:
    return 'D'
  elif nutri_score >= 19 and nutri_score <= 40:
    return 'E'

def nutriScoreBeverages(nutri_score):
  if nutri_score <= 1:
    return 'B'
  elif nutri_score >= 2 and nutri_score <= 5:
    return 'C'
  elif nutri_score >= 6 and nutri_score <= 9:
    return 'D'
  elif nutri_score >= 10 and nutri_score <= 40:
    return 'E'

In [136]:
#add sodium score
target_sodium_id = [3524]
for i in range(len(food)):
  food_id = food['id'].iloc[i]
  index = content_compound[content_compound['food_id'] == food_id].index
  df_sodium = content_compound.loc[index]
  df_sodium = df_sodium[df_sodium['source_id'].isin(target_sodium_id)]
  if len(df_sodium) == 0:
    continue
  score = scoreSodium(df_sodium.orig_content.iloc[0])
  content_compound.at[df_sodium.index, 'score'] = score

In [137]:
target_protein_id = [2]
target_carbohydrate_id = [3]
target_fatty_acics_id = [4]
target_fiber_id = [5]
target_energy = [38]
for i in range(len(food)):
  food_id = food['id'].iloc[i]
  index = content_nutrient[content_nutrient['food_id'] == food_id].index
  df = content_nutrient.loc[index]

  #add protein score
  df_protein = df[df['source_id'].isin(target_protein_id)]
  if len(df_protein) == 0:
    continue
  score = scoreProteins(df_protein.orig_content.iloc[0])
  content_nutrient.at[df_protein.index, 'score'] = score

  #add sugar score
  df_carbohydrate = df[df['source_id'].isin(target_carbohydrate_id)]
  df_fiber = df[df['source_id'].isin(target_fiber_id)]
  if len(df_carbohydrate) == 0:
    continue
  if len(df_fiber) == 0:
    continue
  score = scoreSugar(df_carbohydrate.orig_content.iloc[0], df_fiber.orig_content.iloc[0])
  content_nutrient.at[df_carbohydrate.index, 'score'] = score

  #add fatty acids score
  df_fatty_acids = df[df['source_id'].isin(target_fatty_acics_id)]
  if len(df_fatty_acids) == 0:
    continue
  score = scoreSaturatedFattyAcis(df_fatty_acids.orig_content.iloc[0])
  content_nutrient.at[df_fatty_acids.index, 'score'] = score

  #add fiber score
  df_fiber = df[df['source_id'].isin(target_fiber_id)]
  if len(df_fiber) == 0:
    continue
  score = scoreFibers(df_fiber.orig_content.iloc[0])
  content_nutrient.at[df_fiber.index, 'score'] = score

  #add energy score
  df_energy = df[df['source_id'].isin(target_energy)]
  if len(df_energy) == 0:
    continue
  score = scoreEnergy(df_energy.orig_content.iloc[0])
  content_nutrient.at[df_energy.index, 'score'] = score

In [138]:
#add Nutri-Score
target_nutrient_id = [2,3,4,5,38]
temp_nutrient = content_nutrient[content_nutrient['source_id'].isin(target_nutrient_id)]

target_compound_id = [3524]
temp_compound = content_compound[content_compound['source_id'].isin(target_compound_id)]

for i in range(len(food)):
  food_id = food['id'].iloc[i]
  food_group = food['food_group'].iloc[i]
  food_subgroup = food['food_subgroup'].iloc[i]
  index_nutrient = content_nutrient[content_nutrient['food_id'] == food_id].index
  index_compound = content_compound[content_compound['food_id'] == food_id].index

  temp_index_nutrient = temp_nutrient[temp_nutrient['food_id'] == food_id].index
  temp_index_compound = temp_compound[temp_compound['food_id'] == food_id].index
  df_nutrient = temp_nutrient.loc[temp_index_nutrient]
  df_compound = temp_compound.loc[temp_index_compound]
  df = pd.concat([df_nutrient, df_compound])
  nutri_score = df['score'].sum(axis = 0)

  if food_subgroup == 'Waters':
    nutri_score_category = 'A'
  elif food_group == 'Beverages':
    nutri_score_category = nutriScoreBeverages(nutri_score)
  else:
    nutri_score_category = nutriScoreFoods(nutri_score)

  content_nutrient.at[index_nutrient, 'nutri_score'] = nutri_score
  content_compound.at[index_compound, 'nutri_score'] = nutri_score
  content_nutrient.at[index_nutrient, 'nutri_score_category'] = nutri_score_category
  content_compound.at[index_compound, 'nutri_score_category'] = nutri_score_category

In [139]:
content_nutrient = content_nutrient[content_nutrient['food_name'].notna()]
content_compound = content_compound[content_compound['food_name'].notna()]

In [140]:
content_nutrient.index.name = 'content_id'
content_compound.index.name = 'content_id'
compounds_health_effect.index.name = 'content_id'
compounds_flavor.index.name = 'content_id'

# **Tags**

In [141]:
#mg/day
rdi_sodium = 2000
rdi_potassium = 2000
rdi_calcium = 800
rdi_iron = 14
rdi_phosphorus = 700

In [142]:
target = [3524, 3522, 3514, 16258, 3521]
content_compound[content_compound['source_id'].isin(target)]['orig_unit'].unique()

array(['mg/100g'], dtype=object)

In [143]:
target_sodium_id = [3524]
target_potassium_id = [3522]
target_calcium_id = [3514]
target_iron_id = [16258]
target_phosphorus_id = [3521]

for i in range(len(food)):
  food_id = food['id'].iloc[i]
  index_nutrient = content_nutrient[content_nutrient['food_id'] == food_id].index
  index_compound = content_compound[content_compound['food_id'] == food_id].index
  df = content_compound.loc[index_compound]

  if len(df) == 0:
    continue
  
  df_sodium = df[df['source_id'].isin(target_sodium_id)]
  if len(df_sodium) != 0 and df_sodium['orig_content'].iloc[0]/rdi_sodium > 0.3:
    sodium = 'high_sodium'
  elif len(df_sodium) == 0 or df_sodium['orig_content'].iloc[0]/rdi_sodium <= 0.3: 
    sodium = np.nan

  df_potassium = df[df['source_id'].isin(target_potassium_id)]
  if len(df_potassium) != 0 and df_potassium['orig_content'].iloc[0]/rdi_potassium > 0.3:
    potassium = 'high_potassium'
  elif len(df_potassium) == 0 or df_potassium['orig_content'].iloc[0]/rdi_potassium <= 0.3: 
    potassium = np.nan

  df_calcium = df[df['source_id'].isin(target_calcium_id)]
  if len(df_calcium) != 0 and df_calcium['orig_content'].iloc[0]/rdi_calcium > 0.3:
    calcium = 'high_calcium'
  elif len(df_calcium) == 0 or df_calcium['orig_content'].iloc[0]/rdi_calcium <= 0.3: 
    calcium = np.nan

  df_iron = df[df['source_id'].isin(target_iron_id)]
  if len(df_iron) != 0 and df_iron['orig_content'].iloc[0]/rdi_iron > 0.3:
    iron = 'high_iron'
  elif len(df_iron) == 0 or df_iron['orig_content'].iloc[0]/rdi_iron <= 0.3: 
    iron = np.nan

  df_phosphorus = df[df['source_id'].isin(target_phosphorus_id)]
  if len(df_phosphorus) != 0 and df_phosphorus['orig_content'].iloc[0]/rdi_phosphorus > 0.3:
    phosphorus = 'high_phosphorus'
  elif len(df_phosphorus) == 0 or df_phosphorus['orig_content'].iloc[0]/rdi_phosphorus <= 0.3: 
    phosphorus = np.nan

  content_nutrient.at[index_nutrient, 'sodium'] = sodium
  content_compound.at[index_compound, 'sodium'] = sodium
  content_nutrient.at[index_nutrient, 'potassium'] = potassium
  content_compound.at[index_compound, 'potassium'] = potassium
  content_nutrient.at[index_nutrient, 'calcium'] = calcium
  content_compound.at[index_compound, 'calcium'] = calcium
  content_nutrient.at[index_nutrient, 'iron'] = iron
  content_compound.at[index_compound, 'iron'] = iron
  content_nutrient.at[index_nutrient, 'phosphorus'] = phosphorus
  content_compound.at[index_compound, 'phosphorus'] = phosphorus

In [144]:
#mg/day
rdi_vitaminD3 = 0.005
rdi_vitaminEAlpha = 12
rdi_vitaminEGamma = 12
rdi_vitaminEDelta = 12
rdi_vitanimEBeta = 12
rdi_vitaminK2 = 0.075
rdi_vitaminB1 = 1.1
rdi_vitaminB2 = 1.4
rdi_vitaminB5 = 0.006
rdi_vitaminB6 = 1.4
rdi_vitaminB8 = 0.05
rdi_vitaminB11 = 0.2

In [145]:
target = [12735, 565, 2431, 2432, 12574, 1310, 8425, 12163, 8323, 574, 14513, 14507]
content_compound[content_compound['source_id'].isin(target)]['orig_unit'].unique()

array(['mg/100g', 'α-TE', 'uM'], dtype=object)

In [146]:
target_vitaminD3 = [12735]
target_vitaminEAlpha = [565]
target_vitaminEGamma = [2431]
target_vitaminEDelta = [2432]
target_vitaminEBeta = [12574]
target_vitaminK2 = [1310]
target_vitaminB1 = [8425]
target_vitaminB2 = [12163]
target_vitaminB5 = [8323]
target_vitaminB6 = [574]
target_vitaminB8 = [14513]
target_vitaminB11 = [14507]

for i in range(len(food)):
  food_id = food['id'].iloc[i]
  index_nutrient = content_nutrient[content_nutrient['food_id'] == food_id].index
  index_compound = content_compound[content_compound['food_id'] == food_id].index
  df = content_compound.loc[index_compound]

  if len(df) == 0:
    continue
  
  df_vitaminD3 = df[df['source_id'].isin(target_vitaminD3)]
  if len(df_vitaminD3) != 0 and df_vitaminD3['orig_content'].iloc[0]/rdi_vitaminD3 > 0.3:
    vitaminD3 = 'high_vitaminD3'
  elif len(df_vitaminD3) == 0 or df_vitaminD3['orig_content'].iloc[0]/rdi_vitaminD3 <= 0.3: 
    vitaminD3 = np.nan

  df_vitaminEAlpha = df[df['source_id'].isin(target_vitaminEAlpha)]
  if len(df_vitaminEAlpha) != 0 and df_vitaminEAlpha['orig_content'].iloc[0]/rdi_vitaminEAlpha > 0.3:
    vitaminEAlpha = 'high_vitaminEAlpha'
  elif len(df_vitaminEAlpha) == 0 or df_vitaminEAlpha['orig_content'].iloc[0]/rdi_vitaminEAlpha <= 0.3: 
    vitaminEAlpha = np.nan

  df_vitaminEGamma = df[df['source_id'].isin(target_vitaminEGamma)]
  if len(df_vitaminEGamma) != 0 and df_vitaminEGamma['orig_content'].iloc[0]/rdi_vitaminEGamma > 0.3:
    vitaminEGamma = 'high_vitaminEGamma'
  elif len(df_vitaminEGamma) == 0 or df_vitaminEGamma['orig_content'].iloc[0]/rdi_vitaminEGamma <= 0.3: 
    vitaminEGamma = np.nan

  df_vitaminEDelta = df[df['source_id'].isin(target_vitaminEDelta)]
  if len(df_vitaminEDelta) != 0 and df_vitaminEDelta['orig_content'].iloc[0]/rdi_vitaminEDelta > 0.3:
    vitaminEDelta = 'high_vitaminEDelta'
  elif len(df_vitaminEDelta) == 0 or df_vitaminEDelta['orig_content'].iloc[0]/rdi_vitaminEDelta <= 0.3: 
    vitaminEDelta = np.nan

  df_vitaminEBeta = df[df['source_id'].isin(target_vitaminEBeta)]
  if len(df_vitaminEBeta) != 0 and df_vitaminEBeta['orig_content'].iloc[0]/rdi_vitanimEBeta > 0.3:
    vitaminEBeta = 'high_vitaminEBeta'
  elif len(df_vitaminEBeta) == 0 or df_vitaminEBeta['orig_content'].iloc[0]/rdi_vitanimEBeta <= 0.3: 
    vitaminEBeta = np.nan

  df_vitaminK2 = df[df['source_id'].isin(target_vitaminK2)]
  if len(df_vitaminK2) != 0 and df_vitaminK2['orig_content'].iloc[0]/rdi_sodium > 0.3:
    vitaminK2 = 'high_vitaminK2'
  elif len(df_vitaminK2) == 0 or df_vitaminK2['orig_content'].iloc[0]/rdi_sodium <= 0.3: 
    vitaminK2 = np.nan

  df_vitaminB1 = df[df['source_id'].isin(target_vitaminB1)]
  if len(df_vitaminB1) != 0 and df_vitaminB1['orig_content'].iloc[0]/rdi_potassium > 0.3:
    vitaminB1 = 'high_vitaminB1'
  elif len(df_vitaminB1) == 0 or df_vitaminB1['orig_content'].iloc[0]/rdi_potassium <= 0.3: 
    vitaminB1 = np.nan

  df_vitaminB2 = df[df['source_id'].isin(target_vitaminB2)]
  if len(df_vitaminB2) != 0 and df_vitaminB2['orig_content'].iloc[0]/rdi_calcium > 0.3:
    vitaminB2 = 'high_vitaminB2'
  elif len(df_vitaminB2) == 0 or df_vitaminB2['orig_content'].iloc[0]/rdi_calcium <= 0.3: 
    vitaminB2 = np.nan

  df_vitaminB5 = df[df['source_id'].isin(target_vitaminB5)]
  if len(df_vitaminB5) != 0 and df_vitaminB5['orig_content'].iloc[0]/rdi_iron > 0.3:
    vitaminB5 = 'high_vitaminB5'
  elif len(df_vitaminB5) == 0 or df_vitaminB5['orig_content'].iloc[0]/rdi_iron <= 0.3: 
    vitaminB5 = np.nan

  df_vitaminB6 = df[df['source_id'].isin(target_vitaminB6)]
  if len(df_vitaminB6) != 0 and df_vitaminB6['orig_content'].iloc[0]/rdi_phosphorus > 0.3:
    vitaminB6 = 'high_vitaminB6'
  elif len(df_vitaminB6) == 0 or df_vitaminB6['orig_content'].iloc[0]/rdi_phosphorus <= 0.3: 
    vitaminB6 = np.nan

  df_vitaminB8 = df[df['source_id'].isin(target_vitaminB8)]
  if len(df_vitaminB8) != 0 and df_vitaminB8['orig_content'].iloc[0]/rdi_iron > 0.3:
    vitaminB8 = 'high_vitaminB8'
  elif len(df_vitaminB8) == 0 or df_vitaminB8['orig_content'].iloc[0]/rdi_iron <= 0.3: 
    vitaminB8 = np.nan

  df_vitaminB11 = df[df['source_id'].isin(target_vitaminB11)]
  if len(df_vitaminB11) != 0 and df_vitaminB11['orig_content'].iloc[0]/rdi_phosphorus > 0.3:
    vitaminB11 = 'high_vitaminB11'
  elif len(df_vitaminB11) == 0 or df_vitaminB11['orig_content'].iloc[0]/rdi_phosphorus <= 0.3: 
    vitaminB11 = np.nan

  content_nutrient.at[index_nutrient, 'vitaminD3'] = vitaminD3
  content_compound.at[index_compound, 'vitaminD3'] = vitaminD3
  content_nutrient.at[index_nutrient, 'vitaminEAlpha'] = vitaminEAlpha
  content_compound.at[index_compound, 'vitaminEAlpha'] = vitaminEAlpha
  content_nutrient.at[index_nutrient, 'vitaminEGamma'] = vitaminEGamma
  content_compound.at[index_compound, 'vitaminEGamma'] = vitaminEGamma
  content_nutrient.at[index_nutrient, 'vitaminEDelta'] = vitaminEDelta
  content_compound.at[index_compound, 'vitaminEDelta'] = vitaminEDelta
  content_nutrient.at[index_nutrient, 'vitaminEBeta'] = vitaminEBeta
  content_compound.at[index_compound, 'vitaminEBeta'] = vitaminEBeta
  content_nutrient.at[index_nutrient, 'vitaminK2'] = vitaminK2
  content_compound.at[index_compound, 'vitaminK2'] = vitaminK2
  content_nutrient.at[index_nutrient, 'vitaminB1'] = vitaminB1
  content_compound.at[index_compound, 'vitaminB1'] = vitaminB1
  content_nutrient.at[index_nutrient, 'vitaminB2'] = vitaminB2
  content_compound.at[index_compound, 'vitaminB2'] = vitaminB2
  content_nutrient.at[index_nutrient, 'vitaminB5'] = vitaminB5
  content_compound.at[index_compound, 'vitaminB5'] = vitaminB5
  content_nutrient.at[index_nutrient, 'vitaminB6'] = vitaminB6
  content_compound.at[index_compound, 'vitaminB6'] = vitaminB6
  content_nutrient.at[index_nutrient, 'vitaminB8'] = vitaminB8
  content_compound.at[index_compound, 'vitaminB8'] = vitaminB8
  content_nutrient.at[index_nutrient, 'vitaminB11'] = vitaminB11
  content_compound.at[index_compound, 'vitaminB11'] = vitaminB11

In [147]:
content_compound.to_csv('content_compound.csv')
content_nutrient.to_csv('content_nutrient.csv')
compounds_health_effect.to_csv('compounds_health_effect.csv')
compounds_flavor.to_csv('compounds_flavor.csv')

In [148]:
from google.colab import files
files.download('content_compound.csv')
files.download('content_nutrient.csv')
files.download('compounds_health_effect.csv')
files.download('compounds_flavor.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>